# 属性描述符

In [ ]:
# 描述符是实现了特定协议的类，这个协议包括__get__、__set__和__delete__方法
# 描述符的用法是，创建一个实例，作为另一个类的类属性

In [ ]:
# 描述符类 - 实现描述符协议的类
# 托管类 - 把描述符实例声明为类属性的类
# 描述符实例 - 描述符类的各个实例
# 托管实例 - 托管类的实例
# 储存属性 - 托管实例中存储自身托管属性的属性
# 托管属性 - 托管类中描述符实例处理的公开属性

In [ ]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name

    # self 就是描述符实例（LineItem.weight/price）
    # instance 是托管实例 (LineItem 实例)
    # value 是要设定的值
    def __set__(self, instance, value):
        if value > 0:
            if value > 0:
                instance.__dict__[self.storage_name] = value
            else:
                raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [ ]:
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
              return getattr(instance, self.storage_name)

    def __set__(self, instancc, value):
        if value > 0:
            setattr(instance, self.storage_name)
        else:
            raise ValueError('value must be > 0')

In [ ]:
# 特性工厂函数
def quantity():
    try:
        quantity.counter += 1
    except AttributeError:
        quantity.coiunter = 0

    storage_name = '_{}:{}'.format('quantity', quantity.counter)

    def qty_getter(instance):
        return getattr(instance, storage_name)

    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')
             

In [ ]:
# 一种新的描述符
import abc

class AutoStorage:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

class Validated(abc.ABC, AutoStorage):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)

    @abc.abstractclassmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""

class Quantity(Validated):
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value

class NonBlank(validated):
    """a string with at least one non-space character"""
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [1]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ','.join(display(x) for x in args)
    return '-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args)

In [12]:
print_args('get', obj, obj, Managed)

'-> Managed.__get__(<Managed object>, <Managed object>, <class Managed>)'

In [5]:
# 覆盖型描述符
class Overriding:
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

# 覆盖型描述符
class OverridingNoGet:
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

# 非覆盖型描述符
class NonOverriding:
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):
        print ('-> Managed.spam({})'.format(display(self)))

In [ ]:
import collections

class Text(collections.UserString):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]